In [ ]:
#REAL MODEL

import cv2
import glob
import pandas as pd #OPTIONAL: imported for convenience
import matplotlib.pyplot as plt #OPTIONAL: imported for convenience
import numpy as np
import os
import keyboard 

camera = cv2.VideoCapture(0)
i = 0
a = 0

#making a folder to store each frame as it gets processed
duplicate = -1
while True:
    duplicate += 1
    path = "C:/Users/Pralhad/Downloads/"
    os.chdir(path)
    if duplicate >= 1:
        folder = 'camera_feed'+'('+ str(duplicate) +')'
    else:
        folder = 'camera_feed'
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.chdir(path+folder+'/')
        cam_feed = folder
        break

        
#making a folder for storing picture snippets images
duplicated = -1
while True:
    duplicated += 1
    path = "C:/Users/Pralhad/Downloads/"
    os.chdir(path)
    if duplicated >= 1:
        folder = 'model_captured_pics'+'('+ str(duplicated) +')'
    else:
        folder = 'model_captured_pics'
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.chdir(path+folder+'/')
        model_capt_pics = folder
        break
        
while True: #since it would be applied to a 24/7 CCTV an Infinite Loop was created.
            #For loop with a range equal to the number of frames that are to be processed can be created for testing purp
        
    ret,frame = camera.read()
    a += 1
    if np.all(frame) == None:
        break
    if a == 30: # number can be manipulated to change the frequency at which each frame is processed
                #value set at 30 as approximately 30 frames are present within 1 second of the video
                #set value to 1 to process each frame
        a = 0
        i+=1
        
        os.chdir("C:/Users/Pralhad/Downloads/"+cam_feed+'/') #must be replaced with a system appropriate path
                                                            #camera_feed must be an empty folder
        cv2.imwrite('camera_feed.jpg',frame)


        #reading original images
        real_image = cv2.imread('C:/Users/Pralhad/Downloads/'+cam_feed+'/'+'camera_feed.jpg') #must be replaced with a system appropriate path

        original_image = cv2.imread('C:/Users/Pralhad/Downloads/'+cam_feed+'/'+'camera_feed.jpg') #must be replaced with a system appropriate path


        #reading the image in greyscale
        grey = cv2.imread('C:/Users/Pralhad/Downloads/'+cam_feed+'/'+'camera_feed.jpg',0)#must be replaced with a system appropriate path

        #resizing the images to an appropriate value
        d = (grey.shape[1] + grey.shape[0])/1000
        if d >= 1:
            grey = cv2.resize(grey,(int(grey.shape[1]/d),int(grey.shape[0]/d)))
            real_image = cv2.resize(real_image,(int(real_image.shape[1]/d),int(real_image.shape[0]/d)))
            original_image = cv2.resize(original_image,(int(original_image.shape[1]/d),int(original_image.shape[0]/d)))


        #convolution filter application
        kernel = np.array(([0,-2,0],[-2,8,-2],[0,-2,0]),np.float32)
        grey = cv2.filter2D(grey,-1,kernel)

        (T,gray) = cv2.threshold(grey,0,255,cv2.THRESH_OTSU)


        #contour detection
        total_pictures = []

        (cnts,_) = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for c in cnts:
            (x,y,w,h) = cv2.boundingRect(c)
            image_dimensions = real_image.shape[0]*real_image.shape[1] 
            if w*h >= 75:
                if w >= h:
                    square = w
                else:
                    square = h
                if square < 50:
                    square = 50
                if x+50 <= real_image.shape[1] and y + 50 <= real_image.shape[0]:
                    cv2.rectangle(original_image, (x, y), (x+square, y+square), (255,255,0), 1)
                    pic = real_image[x:x+square,y:y+square]
                    if len(pic) != 0:       
                        total_pictures.append(pic)

        b = 0
        for image in total_pictures:
            b = b + 1
            os.chdir("C:/Users/Pralhad/Downloads/"+model_capt_pics+'/')
            cv2.imwrite( "image"+str(b)+'.jpg',image)



        from keras.models import load_model
        from PIL import Image, ImageOps
        import numpy as np
        import os
        import glob


        os.chdir('C:/Users/Pralhad/Downloads/')
        # Load the model
        model = load_model('keras_model.h5')

        # Create the array of the right shape to feed into the keras model
        # The 'length' or number of images you can put into the array is
        # determined by the first position in the shape tuple, in this case 1.
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        # Replace this with the path to your image
        os.chdir('C:/Users/Pralhad/Downloads/'+model_capt_pics+'/')
        jpg_files = glob.glob('*.jpg')
        roi = 0
        for file in jpg_files[0:len(total_pictures)+1]:
            roi += 1
            image = Image.open('C:/Users/Pralhad/Downloads/'+model_capt_pics+'/'+file)
            #resize the image to a 224x224 with the same strategy as in TM2:
            #resizing the image to be at least 224x224 and then cropping from the center
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.ANTIALIAS)

            #turn the image into a numpy array
            image_array = np.asarray(image)
            # Normalize the image
            normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
            # Load the image into the array
            data[0] = normalized_image_array

            # run the inference
            prediction = model.predict(data)
            if jpg_files.index(file) == 0:
                print('---------------------------IMAGE '+str(i)+'--------------------------------------------------')
            print('REGION OF INTEREST',str(roi),':')
            print("Emergency Vehicle: ",str(prediction[0][0]*100)+'%')
            print('Non-Emergency Vehicle :',str(prediction[0][1]*100+prediction[0][2]*100+prediction[0][3]*100+prediction[0][4]*100)+'%\n')
            if prediction[0][0]*100 > 50:
                print('GREEN SIGNAL OVERRIDE')
                break



C:\Users\Pralhad\Downloads\camera_feed
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
---------------------------FRAME 1--------------------------------------------------

Emergency Vehicle:  81.42678141593933%
 Car:  81.42678141593933%
 Auto-Rickshaw:  0.851944275200367%
 Truck:  13.269507884979248%
 JCB:  3.9078444242477417%


Emergency Vehicle:  0.00012432141147655784%
 Car:  0.00012432141147655784%
 Auto-Rickshaw:  0.00025267584078392247%
 Truck:  99.996018409729%
 JCB:  0.0006377454155881424%


Emergency Vehicle:  1.5212796711239207e-05%
 Car:  1.5212796711239207e-05%
 Auto-Rickshaw:  0.0222521586692892%
 Truck:  99.97461438179016%
 JCB:  0.00020114214294153498%


Emergency Vehicle:  0.017785068484954536%
 Car:  0.017785068484954536%
 Auto-Rickshaw:  0.007142466347431764%
 Truck:  99.96968507766724%
 JCB:  0.0051276172598591074%


Emergency Vehicle:  0.0015729410733911209%
 Car:  0.0015729410733911209%
 Auto-Rickshaw:  0.0045610933739


Emergency Vehicle:  0.001713208075670991%
 Car:  0.001713208075670991%
 Auto-Rickshaw:  0.008779470954323187%
 Truck:  99.98389482498169%
 JCB:  0.004482049916987307%


Emergency Vehicle:  0.004887635077466257%
 Car:  0.004887635077466257%
 Auto-Rickshaw:  0.0200465670786798%
 Truck:  99.96007084846497%
 JCB:  0.012788378808181733%


Emergency Vehicle:  0.0021077637939015403%
 Car:  0.0021077637939015403%
 Auto-Rickshaw:  0.004518395508057438%
 Truck:  99.9893069267273%
 JCB:  0.0036055822420166805%


Emergency Vehicle:  0.0005612717814074131%
 Car:  0.0005612717814074131%
 Auto-Rickshaw:  0.0037935900763841346%
 Truck:  99.99308586120605%
 JCB:  0.0021543895854847506%


Emergency Vehicle:  0.002996418515976984%
 Car:  0.002996418515976984%
 Auto-Rickshaw:  0.029929759330116212%
 Truck:  99.95848536491394%
 JCB:  0.006648225098615512%


Emergency Vehicle:  0.009039472934091464%
 Car:  0.009039472934091464%
 Auto-Rickshaw:  0.004019306288682856%
 Truck:  99.98117089271545%
 JCB:  0.005

 Truck:  99.99206066131592%
 JCB:  0.0011787308721977752%


Emergency Vehicle:  0.0004577437721309252%
 Car:  0.0004577437721309252%
 Auto-Rickshaw:  0.0023129234250518493%
 Truck:  99.99632835388184%
 JCB:  0.0007547236236860044%


Emergency Vehicle:  0.006163275247672573%
 Car:  0.006163275247672573%
 Auto-Rickshaw:  0.00785903466749005%
 Truck:  99.9772846698761%
 JCB:  0.00807198157417588%


Emergency Vehicle:  0.012821200652979314%
 Car:  0.012821200652979314%
 Auto-Rickshaw:  0.0166354700922966%
 Truck:  99.9581515789032%
 JCB:  0.011867255670949817%


Emergency Vehicle:  0.00543302194273565%
 Car:  0.00543302194273565%
 Auto-Rickshaw:  0.02003520494326949%
 Truck:  99.96393918991089%
 JCB:  0.009433341620024294%


Emergency Vehicle:  0.0011786896720877849%
 Car:  0.0011786896720877849%
 Auto-Rickshaw:  0.006748391024302691%
 Truck:  99.98666048049927%
 JCB:  0.004480143979890272%


Emergency Vehicle:  0.005525916640181094%
 Car:  0.005525916640181094%
 Auto-Rickshaw:  0.01156629

 Auto-Rickshaw:  0.005453304765978828%
 Truck:  99.98980760574341%
 JCB:  0.00347874010913074%


Emergency Vehicle:  0.006503715121652931%
 Car:  0.006503715121652931%
 Auto-Rickshaw:  0.013103167293593287%
 Truck:  99.97403025627136%
 JCB:  0.005580334618571214%


Emergency Vehicle:  0.018044862372335047%
 Car:  0.018044862372335047%
 Auto-Rickshaw:  0.018805051513481885%
 Truck:  99.95301961898804%
 JCB:  0.009806331217987463%


Emergency Vehicle:  0.035360935726203024%
 Car:  0.035360935726203024%
 Auto-Rickshaw:  0.011674574489006773%
 Truck:  99.94091987609863%
 JCB:  0.011578321573324502%


Emergency Vehicle:  1.5918098390102386%
 Car:  1.5918098390102386%
 Auto-Rickshaw:  0.19763018935918808%
 Truck:  98.07553887367249%
 JCB:  0.12374302605167031%


Emergency Vehicle:  0.0032414827728644013%
 Car:  0.0032414827728644013%
 Auto-Rickshaw:  0.0042286585085093975%
 Truck:  99.98879432678223%
 JCB:  0.0033806401916081086%


Emergency Vehicle:  0.005233128467807546%
 Car:  0.005233128

 Truck:  99.96126294136047%
 JCB:  0.013368960935622454%


Emergency Vehicle:  0.0015954306945786811%
 Car:  0.0015954306945786811%
 Auto-Rickshaw:  0.017565739108249545%
 Truck:  99.97314810752869%
 JCB:  0.005889315070817247%


Emergency Vehicle:  0.006827338074799627%
 Car:  0.006827338074799627%
 Auto-Rickshaw:  0.006401701830327511%
 Truck:  99.98063445091248%
 JCB:  0.005728507676394656%


Emergency Vehicle:  0.0067803637648466974%
 Car:  0.0067803637648466974%
 Auto-Rickshaw:  0.032236718107014894%
 Truck:  99.94674324989319%
 JCB:  0.012410632916726172%


Emergency Vehicle:  0.001868909203039948%
 Car:  0.001868909203039948%
 Auto-Rickshaw:  0.008891705510905012%
 Truck:  99.98329877853394%
 JCB:  0.004939549035043456%


Emergency Vehicle:  0.0010549866601650137%
 Car:  0.0010549866601650137%
 Auto-Rickshaw:  0.00974393478827551%
 Truck:  99.98388290405273%
 JCB:  0.004358688238426112%


Emergency Vehicle:  0.001885691199277062%
 Car:  0.001885691199277062%
 Auto-Rickshaw:  0.0

...


Emergency Vehicle:  5.011304095387459%
 Car:  5.011304095387459%
 Auto-Rickshaw:  0.4167154897004366%
 Truck:  94.3035364151001%
 JCB:  0.2475860295817256%


Emergency Vehicle:  0.0007208495844679419%
 Car:  0.0007208495844679419%
 Auto-Rickshaw:  0.008184448961401358%
 Truck:  99.98633861541748%
 JCB:  0.003866093175020069%


Emergency Vehicle:  8.779629752098117e-05%
 Car:  8.779629752098117e-05%
 Auto-Rickshaw:  0.0044520293158711866%
 Truck:  99.99352693557739%
 JCB:  0.0012706567758868914%


Emergency Vehicle:  0.0019166474885423668%
 Car:  0.0019166474885423668%
 Auto-Rickshaw:  0.018834133516065776%
 Truck:  99.97422099113464%
 JCB:  0.004224253643769771%


Emergency Vehicle:  0.00020437428247532807%
 Car:  0.00020437428247532807%
 Auto-Rickshaw:  0.003078204463236034%
 Truck:  99.99488592147827%
 JCB:  0.001535865339974407%


Emergency Vehicle:  0.00036764092783414526%
 Car:  0.00036764092783414526%
 Auto-Rickshaw:  0.01264239544980228%
 Truck:  99.98365640640259%
 JCB:  0.002


Emergency Vehicle:  4.268997884082637e-05%
 Car:  4.268997884082637e-05%
 Auto-Rickshaw:  0.0045246441004564986%
 Truck:  99.9943733215332%
 JCB:  0.0007404573807434645%


Emergency Vehicle:  0.0004098349108971888%
 Car:  0.0004098349108971888%
 Auto-Rickshaw:  0.005788526686956175%
 Truck:  99.99055862426758%
 JCB:  0.0026741547117126174%


Emergency Vehicle:  0.02506613964214921%
 Car:  0.02506613964214921%
 Auto-Rickshaw:  0.005641245297738351%
 Truck:  99.95681643486023%
 JCB:  0.011902198457391933%


Emergency Vehicle:  1.223472785204649%
 Car:  1.223472785204649%
 Auto-Rickshaw:  0.23984939325600863%
 Truck:  98.42382073402405%
 JCB:  0.10226437589153647%


Emergency Vehicle:  0.0002630931021485594%
 Car:  0.0002630931021485594%
 Auto-Rickshaw:  0.006535584543598816%
 Truck:  99.99040365219116%
 JCB:  0.0021683645172743127%


Emergency Vehicle:  0.00011134693522762973%
 Car:  0.00011134693522762973%
 Auto-Rickshaw:  0.004360741877462715%
 Truck:  99.99293088912964%
 JCB:  0.0018



Emergency Vehicle:  0.0008431527021457441%
 Car:  0.0008431527021457441%
 Auto-Rickshaw:  0.0036631725379265845%
 Truck:  99.9929428100586%
 JCB:  0.0022021786207915284%


Emergency Vehicle:  0.03153952129650861%
 Car:  0.03153952129650861%
 Auto-Rickshaw:  0.060710590332746506%
 Truck:  99.88377690315247%
 JCB:  0.019567261915653944%


Emergency Vehicle:  0.05113353836350143%
 Car:  0.05113353836350143%
 Auto-Rickshaw:  0.00875810073921457%
 Truck:  99.92979764938354%
 JCB:  0.009854954259935766%


Emergency Vehicle:  0.0007993911822268274%
 Car:  0.0007993911822268274%
 Auto-Rickshaw:  0.004085638647666201%
 Truck:  99.99221563339233%
 JCB:  0.0024928225684561767%


Emergency Vehicle:  0.0008989328307507094%
 Car:  0.0008989328307507094%
 Auto-Rickshaw:  0.006128363020252436%
 Truck:  99.98971223831177%
 JCB:  0.002786237200780306%


Emergency Vehicle:  0.0009277970093535259%
 Car:  0.0009277970093535259%
 Auto-Rickshaw:  0.014432125317398459%
 Truck:  99.97863173484802%
 JCB:  0.0

C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
---------------------------FRAME 4--------------------------------------------------

Emergency Vehicle:  99.99867677688599%
 Car:  99.99867677688599%
 Auto-Rickshaw:  0.0001273626025977137%
 Truck:  0.0009223133929481264%
 JCB:  0.0002712255763981375%

GREEN SIGNAL OVERRIDE
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
---------------------------FRAME 5--------------------------------------------------

Emergency Vehicle:  99.98748302459717%
 Car:  99.98748302459717%
 Auto-Rickshaw:  8.288358799291018e-05%
 Truck:  0.01132484758272767%
 JCB:  0.001109453569370089%

GREEN SIGNAL OVERRIDE
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
---------------------------F

KeyboardInterrupt: 

In [ ]:
#interrupt the previous cell and run this code in order to stop the code and switch the camera off respectively
camera.release()

In [ ]:
#TEST MODEL

import cv2
import glob
import pandas as pd #OPTIONAL: imported for convenience
import matplotlib.pyplot as plt #OPTIONAL: imported for convenience
import numpy as np
import os
import keyboard #OPTIONAL: incase you want to stop sharing camera after a certain frame number

camera = cv2.VideoCapture('C:/Users/Pralhad/Downloads/traffic.mp4') #a video file of traffic

#making a folder to store each frame as it gets processed
duplicate = -1
while True:
    duplicate += 1
    path = "C:/Users/Pralhad/Downloads/"
    os.chdir(path)
    if duplicate >= 1:
        folder = 'camera_feed'+'('+ str(duplicate) +')'
    else:
        folder = 'camera_feed'
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.chdir(path+folder+'/')
        cam_feed = folder
        break

        
#making a folder for storing picture snippets images
duplicated = -1
while True:
    duplicated += 1
    path = "C:/Users/Pralhad/Downloads/"
    os.chdir(path)
    if duplicated >= 1:
        folder = 'model_captured_pics'+'('+ str(duplicated) +')'
    else:
        folder = 'model_captured_pics'
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.chdir(path+folder+'/')
        model_capt_pics = folder
        break
i = 0
a = 0
while True: #since it would be applied to a 24/7 CCTV an Infinite Loop was created.
            #For loop with a range equal to the number of frames that are to be processed can be created for testing purposes
    ret,frame = camera.read()
    a+=1
    if np.all(frame) == None:
        break
    if a == 30:
        a = 0
        i+=1
        os.chdir("C:/Users/Pralhad/Downloads/"+ cam_feed + '/') #must be replaced with a system appropriate path
        cv2.imwrite('camera_feed.jpg',frame)


        #reading original images
        real_image = cv2.imread('C:/Users/Pralhad/Downloads/'+ cam_feed + '/'+'camera_feed.jpg') #must be replaced with a system appropriate path
        original_image = cv2.imread('C:/Users/Pralhad/Downloads/'+ cam_feed + '/'+'camera_feed.jpg') #must be replaced with a system appropriate path


        #reading the image in greyscale
        grey = cv2.imread('C:/Users/Pralhad/Downloads/'+ cam_feed + '/'+'camera_feed.jpg',0)#must be replaced with a system appropriate path

        #resizing the images to an appropriate value
        d = (grey.shape[1] + grey.shape[0])/1000
        if d >= 1:
            grey = cv2.resize(grey,(int(grey.shape[1]/d),int(grey.shape[0]/d)))
            real_image = cv2.resize(real_image,(int(real_image.shape[1]/d),int(real_image.shape[0]/d)))
            original_image = cv2.resize(original_image,(int(original_image.shape[1]/d),int(original_image.shape[0]/d)))

        #convolution filter application
        kernel = np.array(([0,-2,0],[-2,8,-2],[0,-2,0]),np.float32)
        grey = cv2.filter2D(grey,-1,kernel)

        (T,gray) = cv2.threshold(grey,0,255,cv2.THRESH_OTSU)


        #contour detection
        total_pictures = []

        (cnts,_) = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for c in cnts:
            (x,y,w,h) = cv2.boundingRect(c)
            image_dimensions = real_image.shape[0]*real_image.shape[1] 
            if w*h >= 75:
                if w >= h:
                    square = w
                else:
                    square = h
                if square < 50:
                    square = 50
                if x+50 <= real_image.shape[1] and y + 50 <= real_image.shape[0]:
                    cv2.rectangle(original_image, (x, y), (x+square, y+square), (255,255,0), 1)
                    pic = real_image[x:x+square,y:y+square]
                    if len(pic) != 0:       
                        total_pictures.append(pic)

        b = 0
        for image in total_pictures:
            b = b + 1
            os.chdir('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/')
            cv2.imwrite( "image"+str(b)+'.jpg',image)



        from keras.models import load_model
        from PIL import Image, ImageOps
        import numpy as np
        import os
        import glob

        os.chdir('C:/Users/Pralhad/Downloads/')
        # Load the model
        model = load_model('keras_model.h5')

        # Create the array of the right shape to feed into the keras model
        # The 'length' or number of images you can put into the array is
        # determined by the first position in the shape tuple, in this case 1.
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        # Replace this with the path to your image
        os.chdir('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/')
        jpg_files = glob.glob('*.jpg')
        roi = 0 
        for file in jpg_files[0:len(total_pictures)+1]:
            roi += 1
            image = Image.open('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/'+file)
            #resize the image to a 224x224 with the same strategy as in TM2:
            #resizing the image to be at least 224x224 and then cropping from the center
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.ANTIALIAS)

            #turn the image into a numpy array
            image_array = np.asarray(image)
            # Normalize the image
            normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
            # Load the image into the array
            data[0] = normalized_image_array

            # run the inference
            prediction = model.predict(data)
            if jpg_files.index(file) == 0:
                print('---------------------------IMAGE '+str(i)+'--------------------------------------------------')
            print('REGION OF INTEREST',str(roi),':')
            print("Emergency Vehicle: ",str(prediction[0][0]*100)+'%')
            print('Non-Emergency Vehicle :',str(prediction[0][1]*100+prediction[0][2]*100+prediction[0][3]*100+prediction[0][4]*100)+'%\n')
            if prediction[0][0]*100 > 50:
                print('GREEN SIGNAL OVERRIDE')
                break



C:\Users\Pralhad\Downloads\camera_feed
C:\Users\Pralhad\Downloads\model_captured_pics
C:\Users\Pralhad\Downloads\model_captured_pics
---------------------------FRAME 1--------------------------------------------------
REGION OF INTEREST 1 :
Emergency Vehicle:  17.818574607372284%
Non-Emergency Vehicle : 82.18142483383417%

REGION OF INTEREST 2 :
Emergency Vehicle:  8.639106154441833%
Non-Emergency Vehicle : 91.3608911447227%

REGION OF INTEREST 3 :
Emergency Vehicle:  8.418624848127365%
Non-Emergency Vehicle : 91.5813710540533%

REGION OF INTEREST 4 :
Emergency Vehicle:  11.45731434226036%
Non-Emergency Vehicle : 88.54269310832024%

REGION OF INTEREST 5 :
Emergency Vehicle:  13.169637322425842%
Non-Emergency Vehicle : 86.83035969734192%

REGION OF INTEREST 6 :
Emergency Vehicle:  9.145181626081467%
Non-Emergency Vehicle : 90.85481613874435%

REGION OF INTEREST 7 :
Emergency Vehicle:  14.266249537467957%
Non-Emergency Vehicle : 85.7337512075901%

REGION OF INTEREST 8 :
Emergency Vehicle

KeyboardInterrupt: 

In [ ]:
#TEST MODEL
#if ambulance % > 20 expand image 

import cv2
import glob
import pandas as pd #OPTIONAL: imported for convenience
import matplotlib.pyplot as plt #OPTIONAL: imported for convenience
import numpy as np
import os
import keyboard #OPTIONAL: incase you want to stop sharing camera after a certain frame number


        
#making a folder for storing picture snippets images
duplicated = -1
while True:
    duplicated += 1
    path = "C:/Users/Pralhad/Downloads/"
    os.chdir(path)
    if duplicated >= 1:
        folder = 'model_captured_pics'+'('+ str(duplicated) +')'
    else:
        folder = 'model_captured_pics'
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.chdir(path+folder+'/')
        model_capt_pics = folder
        break
i = 0
a = 0

#reading original images
real_image = cv2.imread('C:/Users/Pralhad/Downloads/fire_truck_detection.jpg') #must be replaced with a system appropriate path

original_image = cv2.imread('C:/Users/Pralhad/Downloads/fire_truck_detection.jpg') #must be replaced with a system appropriate path


#reading the image in greyscale
grey = cv2.imread('C:/Users/Pralhad/Downloads/fire_truck_detection.jpg',0)#must be replaced with a system appropriate path
d = (grey.shape[1] + grey.shape[0])/1000
print(d)
print(grey.shape)
if d >= 1:
    grey = cv2.resize(grey,(int(grey.shape[1]/d),int(grey.shape[0]/d)))
    real_image = cv2.resize(real_image,(int(real_image.shape[1]/d),int(real_image.shape[0]/d)))
    original_image = cv2.resize(original_image,(int(original_image.shape[1]/d),int(original_image.shape[0]/d)))

#convolution filter application
kernel = np.array(([0,-2,0],[-2,8,-2],[0,-2,0]),np.float32)
grey = cv2.filter2D(grey,-1,kernel)
cv2.imshow('gry',grey)

(T,gray) = cv2.threshold(grey,0,255,cv2.THRESH_OTSU)

cv2.imshow('gro',gray)      
#contour detection
total_pictures = []

(cnts,_) = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in cnts:
    (x,y,w,h) = cv2.boundingRect(c)
    image_dimensions = real_image.shape[0]*real_image.shape[1] 
    if w*h >= 75:
        if w >= h:
            square = w
        else:
            square = h
        if square < 50:
            square = 50
        cv2.rectangle(original_image, (x, y), (x+square, y+square), (255,255,0), 1)
        pic = real_image[x:x+square,y:y+square]
        if len(pic) != 0:       
            total_pictures.append(pic)
            
cv2.imshow('original',original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
b = 0

for image in total_pictures:
    b = b + 1
    os.chdir('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/')
    cv2.imwrite( "image"+str(b)+'.jpg',image)
    



from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os
import glob

os.chdir('C:/Users/Pralhad/Downloads/')
# Load the model
model = load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
os.chdir('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/')
jpg_files = glob.glob('*.jpg')
print(len(jpg_files))
print(len(total_pictures))
roi = 0 
index = -1
plt.imshow(real_image)
plt.axis('off')
plt.show()
for file in jpg_files:
    roi += 1
    image = Image.open('C:/Users/Pralhad/Downloads/'+str(model_capt_pics)+'/'+file)
    if roi == 1:
        print(image)
        print(real_image.shape)
    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    index += 1
    prediction = model.predict(data)
    if prediction[0][0]*100 >20:
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        if jpg_files.index(file) == 0:
            print('---------------------------IMAGE '+str(1)+'--------------------------------------------------')
        print('REGION OF INTEREST',str(roi),':')
        print("Emergency Vehicle: ",str(prediction[0][0]*100)+'%')
        print('Non-Emergency Vehicle :',str(prediction[0][1]*100+prediction[0][2]*100+prediction[0][3]*100+prediction[0][4]*100)+'%\n')
        if prediction[0][0]*100 > 50:
            print('GREEN SIGNAL OVERRIDE')
            break
